In [3]:
import pandas as pd
#第一行去掉
loans = pd.read_csv('LoanStats3a.csv', skiprows=1)
half_count = len(loans) / 2
#丢弃含有一半缺失值的列
loans = loans.dropna(thresh=half_count,axis=1)
#删除无效列
loans = loans.drop(['desc','url'],axis=1)
loans.to_csv('loans.csv',index=False)


D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
import pandas as pd
loans = pd.read_csv('loans.csv')
#去掉重复数据
loans.drop_duplicates()
print(loans.iloc[0])
print(loans.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
#去除无关特征
loans = loans.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)
loans = loans.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)
loans = loans.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans.iloc[0])
print(loans.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               N

In [55]:
#统计不同还款状态对应的样本数量
print(loans['loan_status'].value_counts())

Fully Paid                                             33902
Charged Off                                             5658
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Current                                                  201
Late (31-120 days)                                        10
In Grace Period                                            9
Late (16-30 days)                                          5
Default                                                    1
Name: loan_status, dtype: int64


In [56]:
#对Fully Paid为1、Charged Off为0 进行二分类

loans = loans[(loans['loan_status'] == 'Fully Paid') | (loans['loan_status'] == 'Charged Off')]

status_replace = {
    'loan_status' : {
        'Fully Paid': 1,
        'Charged Off': 0,
    }
}

loans = loans.replace(status_replace)

In [57]:
#如果某一列的属性只有一个，则删除这一列
#无效特征过多容易过拟合

orig_columns = loans.columns
drop_columns = []
for col in orig_columns:
    col_series = loans[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans = loans.drop(drop_columns,axis=1)
print(drop_columns)
print(loans.shape)  
loans.to_csv('filtered_loans.csv', index=False)

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']
(39560, 24)


In [1]:
import pandas as pd 
loans = pd.read_csv('filtered_loans.csv')
nulls_counts = loans.isnull().sum()
print(nulls_counts)


loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1073
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
pymnt_plan                 0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


In [2]:
loans = loans.drop('pub_rec_bankruptcies',axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())


object     12
float64    10
int64       1
dtype: int64


# 把object类型处理为数值型

In [3]:
#输出数据类型为字符型的字段
object_columns_df = loans.select_dtypes(include=['object'])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
pymnt_plan                       n
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Nov-2016
Name: 0, dtype: object


In [4]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())


RENT        18371
MORTGAGE    17131
OWN          2827
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16436
Verified           12251
Source Verified     9741
Name: verification_status, dtype: int64
10+ years    8821
< 1 year     4563
2 years      4371
3 years      4074
4 years      3409
5 years      3270
1 year       3227
6 years      2212
7 years      1756
8 years      1472
9 years      1253
Name: emp_length, dtype: int64
 36 months    28234
 60 months    10194
Name: term, dtype: int64
CA    6882
NY    3684
FL    2766
TX    2659
NJ    1814
IL    1480
PA    1470
VA    1371
GA    1352
MA    1306
OH    1177
MD    1030
AZ     828
WA     800
CO     764
NC     754
CT     728
MI     688
MO     658
MN     589
NV     477
SC     462
WI     439
OR     431
AL     429
LA     425
KY     321
OK     292
KS     258
UT     251
AR     233
DC     211
RI     196
NM     182
HI     168
WV     167
NH     162
DE     110
WY      79
MT      78
AK      77
SD      62
VT  

In [6]:
#输出类似特征，保留'purpose'
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    18137
credit_card            4970
other                  3803
home_improvement       2869
major_purchase         2108
small_business         1771
car                    1492
wedding                 932
medical                 667
moving                  557
house                   365
vacation                350
educational             312
renewable_energy         95
Name: purpose, dtype: int64
Debt Consolidation                                             2128
Debt Consolidation Loan                                        1671
Personal Loan                                                   640
Consolidation                                                   503
debt consolidation                                              483
Credit Card Consolidation                                       348
Home Improvement                                                344
Debt consolidation                                              323
Small Business Loan                 

In [63]:
#将"emp_length"字段转化为字符型数据
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
#将百分比类型转化为浮点型（小数类型）
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

In [64]:
#给cat_columns里面的字段进行onehot编码
cat_columns = ["home_ownership", "verification_status", "emp_length", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)
loans = loans.drop("pymnt_plan", axis=1)
loans.to_csv('cleaned_loans.csv', index=False)

In [1]:
import pandas as pd
#读取最终清洗的数据
loans = pd.read_csv("cleaned_loans.csv")
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38428 entries, 0 to 38427
Data columns (total 37 columns):
loan_amnt                              38428 non-null float64
int_rate                               38428 non-null float64
installment                            38428 non-null float64
annual_inc                             38428 non-null float64
loan_status                            38428 non-null int64
dti                                    38428 non-null float64
delinq_2yrs                            38428 non-null float64
inq_last_6mths                         38428 non-null float64
open_acc                               38428 non-null float64
pub_rec                                38428 non-null float64
revol_bal                              38428 non-null float64
revol_util                             38428 non-null float64
total_acc                              38428 non-null float64
home_ownership_MORTGAGE                38428 non-null int64
home_ownership_NONE        

In [2]:
#使用逻辑回归模型对0-1标签进行分类
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict, KFold
lr = LogisticRegression()
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)


kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)
#负例预测为正例
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
# 正例预测为正例
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
# 负例预测为正例
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
# 负例预测为负例
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# 统计指标：tpr和fpr，其中tpr越大越好，fpr越小越好
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))
print(tpr)
print(fpr)
print(predictions[:20])

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9991519520247145
0.997967104047311
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
dtype: int64


In [3]:
#样本不均衡，改变权重
#在正负样本比例均衡条件下训练模型，统计相关指标
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict
lr = LogisticRegression(class_weight="balanced")
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)
#负例预测为正例
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
# 正例预测为正例
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
# 负例预测为正例
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
# 负例预测为负例
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# 统计指标：tpr和fpr，其中tpr越大越好，fpr越小越好
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))
print(tpr)
print(fpr)
print(predictions[:20])

0.6669594451343247
0.39308815376085754
0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     0
8     0
9     0
10    1
11    0
12    1
13    1
14    0
15    0
16    1
17    1
18    1
19    0
dtype: int64


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict
#因为这里负例样本数量远远小于正例样本数量，我们可以调整正负样本的权重，使得正负样本比例均衡
penalty = {
    0: 5,
    1: 1
}
lr = LogisticRegression(class_weight=penalty)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)
#负例预测为正例
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
# 正例预测为正例
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
# 负例预测为正例
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
# 负例预测为负例
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# 统计指标：tpr和fpr，其中tpr越大越好，fpr越小越好
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))
print(tpr)
print(fpr)
print(predictions[:20])

0.7084532210679347
0.463130659767141
0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     0
8     0
9     1
10    1
11    0
12    0
13    1
14    0
15    1
16    1
17    1
18    1
19    0
dtype: int64


In [5]:
#使用随机森林
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict
rf = RandomForestClassifier(n_estimators=10,class_weight="balanced", random_state=1)
#print help(RandomForestClassifier)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)


# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])
# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])
# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])
# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))

print(tpr)
print(fpr)
print(predictions[:20])

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.9744071236029924
0.9414156348179634
0     1
1     0
2     1
3     1
4     1
5     1
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    1
14    0
15    1
16    1
17    1
18    1
19    1
dtype: int64
